## Imports

In [19]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import ast

## Data Read-in

In [2]:
df = pd.read_csv("Untitled spreadsheet - Sheet1 (4).csv")

In [4]:
df['full_address'] = df['Address'] + ' ' + df['State']

In [6]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [9]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

In [11]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [21]:
df

,Pin,Address,State,Sheet_Link,full_address,geocoded,lat,lon
0,20-29-407-017-0000,"7551 S GREEN ST, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"7551 S GREEN ST, CHICAGO Illinois","(41.756256, -87.645031)",41.756256,-87.645031
1,20-25-325-023-0000,"7830 S EAST END AVE, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"7830 S EAST END AVE, CHICAGO Illinois","(41.7523146, -87.5837991)",41.752315,-87.583799
2,20-17-200-008-0000,"1135 W GARFIELD BLVD, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"1135 W GARFIELD BLVD, CHICAGO Illinois","(41.7935367, -87.6538396)",41.793537,-87.653840
3,20-17-227-016-0000,"1003 W 58TH ST, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"1003 W 58TH ST, CHICAGO Illinois","(41.7886812, -87.6501301)",41.788681,-87.650130
4,20-08-318-032-0000,"5451 S LAFLIN ST, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"5451 S LAFLIN ST, CHICAGO Illinois","(41.79439319999999, -87.66195689999999)",41.794393,-87.661957
5,20-26-220-020-0000,"7359 S DORCHESTER AVE, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"7359 S DORCHESTER AVE, CHICAGO Illinois","(41.760705, -87.590491)",41.760705,-87.590491
6,20-31-223-001-0000,"8101 S MARSHFIELD AVE, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"8101 S MARSHFIELD AVE, CHICAGO Illinois","(41.7465957, -87.66428359999999)",41.746596,-87.664284
7,20-28-125-003-0000,"7411 S EGGLESTON AVE, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"7411 S EGGLESTON AVE, CHICAGO Illinois","(41.7594284, -87.6354691)",41.759428,-87.635469
8,20-17-220-018-0000,"5757 S MORGAN ST, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"5757 S MORGAN ST, CHICAGO Illinois","(41.7889451, -87.649491)",41.788945,-87.649491
9,20-22-312-028-0000,"6956 S WABASH AVE, CHICAGO",Illinois,https://docs.google.com/spreadsheets/d/1Gy1dgY...,"6956 S WABASH AVE, CHICAGO Illinois","(41.7675053, -87.62383100000001)",41.767505,-87.623831


In [23]:
# import folium
# from folium.plugins import MarkerCluster
# import pandas as pd
# import textwrap

def tooltip_html(row_index, max_chars=30):
    """
    Generate HTML content for the tooltip with line breaks at word boundaries
    and without any labels.
    
    Parameters:
    - row_index (int): The index of the row in the DataFrame.
    - max_chars (int): Maximum number of characters per line.
    
    Returns:
    - str: HTML string for the tooltip content.
    """
    # Retrieve the address from 'Display Properties'
    address = df.at[row_index, 'Address']
    address_str = str(address).strip() if pd.notnull(address) else 'N/A'
    
    # Use textwrap to wrap text without breaking words
    wrapped_address = '<br>'.join(textwrap.wrap(address_str, width=max_chars, break_long_words=False))
    
    # Construct the tooltip HTML without any labels
    tooltip_content = f'<div class="popup-content">{wrapped_address}</div>'
    return tooltip_content

# Initialize the map centered around the mean latitude and longitude
map_center = ast.literal_eval(df['geocoded'].iloc[0])
m = folium.Map(location=map_center, zoom_start=12)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the tooltips
custom_css = """
<style>
    .popup-content {
        max-width: 200px; /* Adjusts the maximum width of the tooltip */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-tooltip {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
</style>
"""

# Add the custom CSS to the map
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
             <h3 align="center" style="font-size:16px"><b>Mark J. Nussbaum Properties</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    # Create tooltip HTML content
    tooltip_content = tooltip_html(i)
    
    # Create a CircleMarker without the 'icon' parameter
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        tooltip=folium.Tooltip(tooltip_content, sticky=True),
        color='red',
        fill=True,
        fill_color='red',
        radius=9,
        weight=1,          # Thickness of the circle border
        opacity=1,       # Opacity of the circle border
        fill_opacity=1   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m

In [24]:
m.save('index.html')

In [25]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MarkNussbaumMap
